In [13]:
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import spacy
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, ldaseqmodel
import pyLDAvis
import pyLDAvis.gensim
import logging
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
import mysql.connector
import datetime
import numpy as np

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [14]:
conn = mysql.connector.connect(host="ba-isdsclass-dev2.lsu.edu",
                                user="aguil66",
                                passwd="msadatabase",
                                database="msa_schema")

In [15]:
cursor = conn.cursor()

In [16]:
sql = ("Select * from text order by article_date asc;")

In [17]:
db = pd.read_sql(sql,conn)

In [18]:
print(db)

                                             article_url  \
0      https://thehill.com/blogs/twitter-room/other-n...   
1      https://thehill.com/policy/healthcare/194204-o...   
2      https://thehill.com/policy/healthcare/228294-o...   
3      https://thehill.com/policy/healthcare/312343-d...   
4      https://thehill.com/policy/healthcare/311908-h...   
5      https://thehill.com/policy/healthcare/275147-n...   
6      https://thehill.com/policy/healthcare/275191-g...   
7      https://thehill.com/policy/healthcare/275205-n...   
8      https://thehill.com/policy/healthcare/275219-s...   
9      https://thehill.com/policy/healthcare/275273-r...   
10     https://thehill.com/policy/healthcare/275291-s...   
11     https://thehill.com/policy/healthcare/194272-a...   
12     https://thehill.com/blogs/floor-action/house/1...   
13     https://thehill.com/blogs/blog-briefing-room/n...   
14     https://thehill.com/blogs/blog-briefing-room/n...   
15     https://thehill.com/blogs/twitter

[37676 rows x 7 columns]


In [19]:
db['time'] = pd.to_datetime(db.article_date.str.strip())

C:\Users\parke\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\parke\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [20]:
db.tail(10)

,article_url,article_date,article_title,article_text,author,twitter,timestamp,time
37666,http://thetoxicologisttoday.blogspot.com/2013/...,"Wednesday, 31 July 2013",\n1st Survival Summer Course\n,"A few weeks ago I had an idea, based on an app...",Pudget,,2020-02-29 16:38:03,2013-07-31
37667,http://thetoxicologisttoday.blogspot.com/2015/...,"Wednesday, 4 November 2015",\nWhat is the Difference Between Bio and Non-B...,When two forces in the universe secretively co...,Pudget,,2020-02-29 16:37:35,2015-11-04
37668,http://thetoxicologisttoday.blogspot.com/2013/...,"Wednesday, 4 September 2013",\nTerritorial Pissings - removing cat's urine ...,I recently moved to a new house to find out th...,Pudget,,2020-02-29 16:38:02,2013-09-04
37669,http://thetoxicologisttoday.blogspot.com/2018/...,"Wednesday, 6 June 2018",\nThe real survival rates to cancer - Part 2 o...,Following on the first part (see here) where t...,Pudget,,2020-02-29 16:37:13,2018-06-06
37670,http://thetoxicologisttoday.blogspot.com/2015/...,"Wednesday, 7 January 2015",\nGreat science news from a recent past - Part...,2014 was proficient in great science. News of ...,Pudget,,2020-02-29 16:37:49,2015-01-07
37671,http://thetoxicologisttoday.blogspot.com/2018/...,"Wednesday, 7 November 2018",\n'Amar a diferença' - a real case of a wonder...,When I moved from Portugal to England I was ta...,Pudget,,2020-02-29 16:37:12,2018-11-07
37672,http://thetoxicologisttoday.blogspot.com/2020/...,"Wednesday, 8 January 2020",\nOn the management of Paediatric Vesicoureter...,For any parent/carer dealing with a child diag...,Pudget,,2020-02-29 16:37:08,2020-01-08
37673,http://thetoxicologisttoday.blogspot.com/2015/...,"Wednesday, 8 July 2015",\nChris Dodd's opening welcome @ LeSpar AMR Wo...,If you haven't read the previous post please d...,Pudget,,2020-02-29 16:37:38,2015-07-08
37674,http://thetoxicologisttoday.blogspot.com/2015/...,"Wednesday, 8 July 2015",\nLESPAR Interdisciplinary Networking Workshop...,Yesterday I attended the LESPAR Interdisciplin...,Pudget,,2020-02-29 16:37:39,2015-07-08
37675,http://thetoxicologisttoday.blogspot.com/2012/...,"Wednesday, 9 May 2012",\nDiscussing PhD careers outside of academia\n,"I'll be back to work on Monday, my rib is heal...",Pudget,,2020-02-29 16:38:14,2012-05-09


In [21]:
db['year'] = pd.DatetimeIndex(db['time']).year

In [22]:
db.head()

,article_url,article_date,article_title,article_text,author,twitter,timestamp,time,year
0,https://thehill.com/blogs/twitter-room/other-n...,01/01/14 05:22 PM EST,Obama tweets: 'I signed the ACA for kids like ...,President Obama on Wednesday gave a shoutout t...,\n By Mario Trujillo -,,2020-02-29 15:19:44,2014-01-01 17:22:00,2014
1,https://thehill.com/policy/healthcare/194204-o...,01/01/14 06:00 AM EST,"O-Care coverage begins, presenting tests",Today is the day for ObamaCare.President Obam...,\n By Jonathan Easley -,,2020-02-29 15:19:44,2014-01-01 06:00:00,2014
2,https://thehill.com/policy/healthcare/228294-o...,01/01/15 01:30 PM EST,ObamaCare's 2015 challenges,"As ObamaCare enters its fifth year, there are ...",\n By Sarah Ferris -,,2020-02-29 15:14:35,2015-01-01 13:30:00,2015
3,https://thehill.com/policy/healthcare/312343-d...,01/01/17 09:39 AM EST,Incoming Dem rep: Getting rid of ObamaCare wit...,Rep.-elect Lisa Blunt Rochester (D-Del.) said ...,Mallory Shelbourne,,2020-02-29 15:07:32,2017-01-01 09:39:00,2017
4,https://thehill.com/policy/healthcare/311908-h...,01/01/17 10:30 AM EST,"House, Senate headed for clash on Medicare",Senate and House Republicans are headed for a ...,Alexander Bolton,,2020-02-29 15:07:31,2017-01-01 10:30:00,2017


In [23]:
uniqueyears, time_slices = np.unique(db['year'],return_counts=True)

In [24]:
print(np.asarray((uniqueyears, time_slices)).T)

[[2006    1]
 [2007    5]
 [2008    1]
 [2010 2412]
 [2011 3232]
 [2012 3281]
 [2013 4033]
 [2014 4688]
 [2015 3952]
 [2016 3953]
 [2017 5126]
 [2018 3230]
 [2019 3081]
 [2020  681]]


In [25]:
print(time_slices)

[   1    5    1 2412 3232 3281 4033 4688 3952 3953 5126 3230 3081  681]


In [26]:
def sent_to_words(sentences):
    for sent in sentences:
        #sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        #sent = re.sub('\s+', ' ', sent)  # remove newline chars
        #sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = db.article_text.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['president', 'obama', 'on', 'wednesday', 'gave', 'shoutout', 'to', 'the', 'then', 'year', 'old', 'boy', 'who', 'stood', 'beside', 'him', 'while', 'he', 'signed', 'his', 'signature', 'healthcare', 'law', 'in', 'in', 'tweet', 'obama', 'said', 'he', 'signed', 'the', 'law', 'for', 'kids', 'like', 'marcelas', 'owens', 'whose', 'mother', 'died', 'of', 'pulmonary', 'hypertension', 'in', 'part', 'because', 'she', 'went', 'without', 'healthcare', 'insurance', 'signed', 'the', 'aca', 'for', 'kids', 'like', 'marcelas', 'owens', 'he', 'lost', 'his', 'mom', 'bc', 'she', 'couldn', 'afford', 'coverage', 'today', 'millions', 'of', 'americans', 'finally', 'can', 'bomany', 'of', 'the', 'provisions', 'in', 'obamacare', 'including', 'coverage', 'and', 'subsidies', 'for', 'individual', 'who', 'purchased', 'insurance', 'on', 'the', 'online', 'healthcare', 'exchange', 'started', 'wednesday', 'the', 'tweet', 'was', 'signed', 'bo', 'signifying', 'that', 'obama', 'wrote', 'it', 'himself', 'in', 'an', 'intervi

In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



from spacy.lang.en import English

#!python -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load(r'C:\Users\parke\Anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.2.5', disable=['parser', 'ner'])
    # nlp = spacy.load(r'C:\Users\parke\Anaconda3\Lib\site-packages\en_core_web_lg\en_core_web_lg-2.2.5', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

KeyboardInterrupt: 

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

In [ ]:
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

In [17]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus = corpus, 
                                 id2word = id2word, 
                                 time_slice = [2,2,2,2],
                                 num_topics = 1)

C:\Users\parke\Anaconda3\lib\site-packages\gensim\models\ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))


KeyboardInterrupt: 

In [ ]:
import pyLDAvis.gensim
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=8, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

In [40]:
df = pd.read_csv('test2.csv')

In [41]:
print(df)

   DATE   PROJECT_ID                                           ABSTRACT
0   2014      589214  Objective(s): 1: Determine the nutrients in po...
1   2015      673465  This project has four broad objectives, linked...
2   2016      917061  Conservation needs to put people back into the...
3   2017      931981  AbstractThe goal of this program is to develop...
4   2018     1005000  DESCRIPTION (provided by applicant): There is ...


In [42]:
uniqueyears2, time_slices2 = np.unique(df['DATE '],return_counts=True)

In [43]:
print(np.asarray((uniqueyears2, time_slices2)).T)

[[2014    1]
 [2015    1]
 [2016    1]
 [2017    1]
 [2018    1]]


In [44]:
def sent_to_words(sentences):
    for sent in sentences:
        #sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        #sent = re.sub('\s+', ' ', sent)  # remove newline chars
        #sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data2 = df.ABSTRACT.values.tolist()
data_words2 = list(sent_to_words(data2))
print(data_words2[:1])

[['objective', 'determine', 'the', 'nutrients', 'in', 'pollen', 'that', 'promote', 'worker', 'longevity', 'determine', 'the', 'effects', 'of', 'pollen', 'mixtures', 'on', 'worker', 'protein', 'and', 'lipid', 'storesand', 'longevity', 'characterize', 'the', 'chemical', 'composition', 'of', 'pollen', 'mixtures', 'that', 'optimize', 'workerprotein', 'and', 'lipid', 'stores', 'and', 'longevity', 'determine', 'the', 'effects', 'of', 'undigested', 'saccharides', 'in', 'high', 'fructose', 'corn', 'syrup', 'hfcs', 'on', 'worker', 'physiology', 'and', 'longevity', 'identify', 'the', 'saccharides', 'in', 'hfcs', 'determine', 'the', 'effect', 'of', 'saccharides', 'in', 'hfcs', 'on', 'worker', 'physiology', 'and', 'longevity', 'evaluate', 'the', 'effects', 'of', 'supplemental', 'feeding', 'on', 'varroa', 'tolerance', 'queenproduction', 'and', 'foraging', 'activity', 'of', 'honey', 'bee', 'colonies', 'modify', 'the', 'megabee', 'diet', 'by', 'adding', 'chemical', 'components', 'that', 'were', 'iden

In [45]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words2, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words2], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



from spacy.lang.en import English

#!python -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load(r'C:\Users\parke\Anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.2.5', disable=['parser', 'ner'])
    # nlp = spacy.load(r'C:\Users\parke\Anaconda3\Lib\site-packages\en_core_web_lg\en_core_web_lg-2.2.5', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready2 = process_words(data_words2)  # processed Text Data!

In [46]:
# Create Dictionary
id2word2 = corpora.Dictionary(data_ready2)

In [47]:
# Create Corpus: Term Document Frequency
corpus2 = [id2word2.doc2bow(text) for text in data_ready2]

In [58]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus = corpus2, 
                                 id2word = id2word2, 
                                 time_slice = time_slices2,
                                 num_topics = 2)

C:\Users\parke\Anaconda3\lib\site-packages\gensim\models\ldaseqmodel.py:293: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [62]:
ldaseq.print_topic_times(topic=0)

[[('resource', 0.036058831910797534),
  ('stewardship', 0.033113556223825814),
  ('land', 0.024241697761567833),
  ('management', 0.024200476140575616),
  ('people', 0.021302246511092734),
  ('urban', 0.021302244774478073),
  ('practice', 0.01837315748024876),
  ('natural', 0.01836674513199405),
  ('range', 0.015463240505383884),
  ('research', 0.015430549544201385),
  ('steward', 0.012569958507057843),
  ('benefit', 0.012569949878311806),
  ('work', 0.012569913905180415),
  ('landscape', 0.012569910683553214),
  ('public', 0.01256990729265597),
  ('forest', 0.012569905587804586),
  ('organization', 0.012569898828702804),
  ('rural', 0.01256989714518662),
  ('private', 0.012569893886326759),
  ('ecological', 0.012569869347346049)],
 [('resource', 0.03610944457064613),
  ('stewardship', 0.03316113706910026),
  ('land', 0.02426535252017068),
  ('management', 0.02422141209564925),
  ('people', 0.021320381330698437),
  ('urban', 0.021320379604742975),
  ('practice', 0.01838592559625764),
 

In [60]:
import pyLDAvis.gensim
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0, corpus=corpus2)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

C:\Users\parke\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
